In [5]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.stats import theilslopes
import rasterio
from rasterio.transform import from_origin
from pathlib import Path

def calculate_theil_sen_and_save_tif(input_nc_file, aggregation='yearly', min_year=None, max_year=None):
    """
    Calculate Theil-Sen slope for temperature data and save results as GeoTIFF.
    
    Parameters:
    input_nc_file (str): Path to NetCDF file with temperature data.
    aggregation (str): 'yearly', 'seasonal', or 'quarterly'. Default is 'yearly'.
    min_year (int): Optional, minimum year for analysis.
    max_year (int): Optional, maximum year for analysis.
    
    Outputs:
    GeoTIFF files with the slopes, named based on the aggregation and time range.
    """
    ds = xr.open_dataset(input_nc_file, engine='netcdf4')

    if not np.issubdtype(ds['date'].dtype, np.datetime64):
        ds['date'] = pd.to_datetime(ds['date'].values, format='%Y%m%d')

    if aggregation == 'yearly':
        resampled = ds.resample(date='Y').mean()
        process_and_save(resampled, input_nc_file, 'yearly', min_year, max_year)

    elif aggregation == 'seasonal':
        seasons = {
            "DJF": resample_by_months(ds, ['12', '01', '02']),
            "MAM": resample_by_months(ds, ['03', '04', '05']),
            "JJA": resample_by_months(ds, ['06', '07', '08']),
            "SON": resample_by_months(ds, ['09', '10', '11'])
        }
        for season, data in seasons.items():
            process_and_save(data, input_nc_file, f'seasonal_{season}', min_year, max_year)

    elif aggregation == 'quarterly':
        quarters = {
            "JFM": resample_by_months(ds, ['01', '02', '03']),
            "AMJ": resample_by_months(ds, ['04', '05', '06']),
            "JAS": resample_by_months(ds, ['07', '08', '09']),
            "OND": resample_by_months(ds, ['10', '11', '12'])
        }
        for quarter, data in quarters.items():
            process_and_save(data, input_nc_file, f'quarterly_{quarter}', min_year, max_year)

    else:
        raise ValueError("Invalid aggregation. Choose 'yearly', 'seasonal', or 'quarterly'.")

def resample_by_months(ds, months):
    return ds.sel(date=ds['date'].dt.month.isin([int(m) for m in months])).resample(date='Y').mean()

def process_and_save(resampled, input_nc_file, aggregation, min_year=None, max_year=None):
    
    resampled['t2m'] = (resampled['t2m'] - 273.15) * 9 / 5 + 32
    years = resampled['date'].dt.year.values
    start_year = min_year if min_year else years.min()
    end_year = max_year if max_year else years.max()

    year_mask = (years >= start_year) & (years <= end_year)
    t2m_data = resampled['t2m'].values[year_mask, :, :]

    lat, lon = resampled['latitude'].values, resampled['longitude'].values
    slope_array = np.zeros((len(lat), len(lon)), dtype=np.float32)

    for i in range(len(lat)):
        for j in range(len(lon)):
            pixel_time_series = t2m_data[:, i, j]
            if not np.any(np.isnan(pixel_time_series)):
                slope, _, _, _ = theilslopes(pixel_time_series, years[year_mask])
                slope_array[i, j] = slope

    slope_array *= (end_year - start_year)
    transform = from_origin(np.min(lon), np.max(lat), np.abs(lon[1] - lon[0]), np.abs(lat[1] - lat[0]))

    meta = {
        'driver': 'GTiff',
        'height': slope_array.shape[0],
        'width': slope_array.shape[1],
        'count': 1,
        'dtype': 'float32',
        'crs': 'EPSG:4326',
        'transform': transform
    }

    output_file_name = f"{Path(input_nc_file).stem}_theilsen_{aggregation}_{start_year}_{end_year}.tif"
    output_tif = Path(input_nc_file).with_name(output_file_name)

    with rasterio.open(output_tif, 'w', **meta) as dst:
        dst.write(slope_array, 1)

    print(f"GeoTIFF saved as {output_file_name}")

# Example usage:
# calculate_theil_sen_and_save_tif('temperature_data.nc', aggregation='seasonal', min_year=1980)

ds = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\Monthly_single_l\data_0.nc"
calculate_theil_sen_and_save_tif(ds, aggregation='seasonal')

GeoTIFF saved as data_0_theilsen_seasonal_DJF_1940_2024.tif
GeoTIFF saved as data_0_theilsen_seasonal_MAM_1940_2024.tif
GeoTIFF saved as data_0_theilsen_seasonal_JJA_1940_2024.tif
GeoTIFF saved as data_0_theilsen_seasonal_SON_1940_2023.tif
